## Действительно ли помогают неразмеченные данные?

Частичное обучение (semi-supervised learning) предлагает методы работы с выборками, в которых лишь для части объектов известны ответы. В статьях утверждается, что добавление неразмеченных данных позволяет повысить качество работы — давайте выясним, так ли это!

Наверное, проще всего добыть неразмеченные примеры, если речь идёт о работе с текстами или изображениями. Остановимся на текстах.

Будем работать с данными из соревнования Predict closed questions on Stack Overflow: https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data

Нас будет интересовать файл train-sample.csv — загрузите его. Будем решать бинарную задачу: отнесём объект к классу 1, если `OpenStatus == 'open'`, и к классу 0 иначе.

**Задание 1. (5 баллов)**

Загрузите данные и подготовьте выборку. В качестве признаков возьмите TF-IDF по BodyMarkdown с `min_df=10`; про целевую переменную написано выше. Выделите тестовую выборку из 5000 объектов.

In [51]:
import numpy as np
import pandas as pd

In [52]:
data = pd.read_csv("train-sample.csv")
data = data[["BodyMarkdown", "OpenStatus"]]

In [53]:
data_train = data
data_train["OpenStatus"] = (data_train["OpenStatus"] == "open").astype(int)
data_train.head()

,BodyMarkdown,OpenStatus
0,I am building a corpus of indexed sentences in...,1
1,i create a xml document with JAXP and search a...,1
2,What are the adverse effects of having too man...,1
3,I am looking for the vb.net equivalent of this...,0
4,"With Spring-Data, you can use the @Document an...",1


In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = data_train["BodyMarkdown"]
Y = data_train["OpenStatus"]

vectorizer = TfidfVectorizer(
    min_df=10
)

from sklearn.model_selection import train_test_split 
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 5000, random_state = 228)

xtrain = vectorizer.fit_transform(xtrain)
xtest = vectorizer.transform(xtest)


Нас будут интересовать качество (AUC-ROC) в четырёх следующих постановках:
1. Модель обучается только на размеченных данных.
2. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть не пересекается с тестовой выборкой.
3. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть совпадает с тестовой выборкой.
4. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть включает в себя тестовую выборку.

**Задание 1. (5 баллов)**

Проведите эксперименты и сделайте выводы для любого из методов пакета `sklearn.semi_supervised` и для логистической регрессии

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import roc_auc_score
from scipy.sparse import vstack

def evaluate(model, xtrain, ytrain, xtest, ytest):
    model.fit(xtrain, ytrain)
    probs = model.predict_proba(xtest)[:, 1]
    return roc_auc_score(ytest, probs)

clf = LogisticRegression(max_iter=1000)
semi_clf = SelfTrainingClassifier(clf)
rng = np.random.RandomState(228)

# Размеченные
auc1 = evaluate(clf, xtrain, ytrain, xtest, ytest)

# Неразмеченные в трейне
mask_train = rng.rand(len(ytrain)) < 0.5
ytrain_half = ytrain.copy()
ytrain_half[mask_train] = -1
auc2 = evaluate(semi_clf, xtrain, ytrain_half, xtest, ytest)

# Неразмеченные это тест
xtrain_combined = vstack([xtrain, xtest])
ytrain_combined = np.concatenate([ytrain, -1 * np.ones(len(ytest))])
auc3 = evaluate(semi_clf, xtrain_combined, ytrain_combined, xtest, ytest)

# Неразмеченные в трейн + тест
xtrain_half_combined = vstack([xtrain, xtest])
ytrain_half_combined = np.concatenate([ytrain_half, ytest])
auc4 = evaluate(semi_clf, xtrain_half_combined, ytrain_half_combined, xtest, ytest)

print(auc1)
print(auc2)
print(auc3)
print(auc4)


0.8250785511161997
0.8193818047560055
0.8261612145931834
0.8554356988316678


### Self-train

Обучаем на размеченной части, предсказываем неразмеченную, потом обучаем на всех, и предсказываем неразмеченную, повторяем пока не сойдёмся в предскзааниях неразмеченной части